**Sustainable Software Development, block course, March 2021**  
*Scientific Software Center, Institute for Scientific Computing, Dr. Inga Ulusoy*

# Analysis of the data

Imagine you perform a "measurement" of some type and obtain "scientific data". You know what your data represents, but you have only a vague idea how different features in the data are connected, and what information you can extract from the data.

You would start first with going through the data, making sure your data set is complete and that the result is reasonable. Imagine this already happened.

In the next step, you would inspect your data more closely and try to identify structures. That is the step that we are focusing on in this unit.

In the `data` folder, you will find several data files (`*.t` and `*.dat`). These are data files generated through some "new approach" that hasn't been used in your lab before. No previous analysis software exists, and you are going to establish a protocol for this "new approach" and "publish your results".

The data can be grouped into two categories: 
1. data to be analyzed using statistical methods;
2. data to be analyzed using numerical methods.

In your hypothetical lab, you are an "expert" in one particular "method", and your co-worker is an "expert" in the other. Combined these two methods will lead to much more impactful results than if only one of you analyzed the data. Now, the task in this course is to be solved collaboratively with your team member working on one of the analysis approaches, and you working on the other. You will both implement functionality into the same piece of "software", but do so collaboratively through git.

As you do not know yet which analysis is most meaningful for your data, and how to implement it, you will start with a jupyter notebook. You and your team member will work on the same notebook that will be part of a github repository for your project. This is the task for today. Discuss with your team members who will work on the statistical and who on the numerical analysis.

## Step 1

Generate a github repository with the relevant files.

In [ ]:
import LCpackage as LC 

LC.statistical.del_below()

## Step 2

Clone the repository to your local machine.

## Step 3

Start working on task 1 for your analysis approach. 

## Step 4

Create your own branch of the repository and commit your changes to your branch; push to the remote repository.

## Step 5

Open a `pull request` so your team member can review your implementation. Likewise, your team member will ask you to review theirs.

## Step 6

Merge the changes in your branch into `main`. Resolve conflicts.

## Step 7

Repeat working on task; committing and pushing to your previously generated branch or a new branch; open a pull request; merge with main; until you have finished all the tasks in your analysis approach. Delete obsolete branches.

In [ ]:
import pandas as pd 
import numpy as np 


df_ = pd.DataFrame(np.array([[1,1,1],[2,3,4],[3,4,5]]), columns=['col1', 'col2', 'col3'], index=['a', 'b', 'c'])

print(df_)

# Start of the analysis notebook
**Author : Your Name**  
*Date : The date you started working on this*  
*Affiliation : The entity under whose name you are working on this*  

Place the required modules in the top, followed by required constants and global functions.

In [ ]:
# required modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn

In [ ]:
# constants and global functions
threshv = 1.0e-5
filenames = 'efield.t' , 'expec.t', 'npop.t', 'nstate_i.t', 'table.dat' #names of the files to acces later
filedir= '../data/'  #directory in which the data is stored

In [ ]:
# reading of the data files
def read_in_df(filedir, filename):
    name = '{}{}'.format(filedir, filename)     #what is this? -> creates filedir-filename "merge"
    print('Reading from file {} - pandas'.format(name))    
    data = pd.read_csv(name, r'\s+')                       #turn csv into dataframe?     sad
    return data
def read_in_np(filedir, filename):
    name = '{}{}'.format(filedir, filename)
    print('Reading from file {} - numpy'.format(name))
    data = np.loadtxt(name, skiprows=1)
    data = data.T
    return data

# Statistical analysis

Find correlations in the data sets. Analyse the data statistically and plot your results.  

Here we would want to do everything with pandas and leave the data in a dataframe. The files that are relevant to you are `expect.t`, `npop.t` and `table.dat`.

### Task 1: Read in expec.t and plot relevant data

In [ ]:
# read and plot expec.t
df_expec = read_in_df(filedir, filenames[1])
#print(df_expec.head(10))               
sn.relplot(data=df_expec, kind='line', x='time', y='<z>')
plt.show()

We can discard the entries norm, \<x>, and \<y> as these are mostly constant.

In [ ]:
# eliminate columns based on the variance: if the variance of the values
# in a column is below a given threshold, that column is discarded

def del_below(df,threshhold):
    dellist=[]                              #create a list which will hold the columns that have a variance below the threshhold
    for i in range(len(df.columns)):        #iterate over all columns
        if df.var()[i] < threshhold:        #identify the relevant columns to delete
            dellist.append(i)               #add their index to the list
    df.drop(df.columns[dellist], axis=1, inplace=True)      #delete the columns
    return df                                                   

df_expec=del_below(df_expec,threshv)


### Task 2: Create plots of the relevant data and save as .pdf.

In [ ]:
# create plots -> just same plot as above?
from pathlib import Path

def save(df, xaxis, yaxis, path):                        
    sn.relplot(data=df, kind='line', x=xaxis, y=yaxis)   
    if Path(path).exists() == False:   
        plt.savefig(path)
    elif Path(path).exists() == True:
        print('{} already exists. Overwrite? Y/N'.format(path))
        check = input()
        if check == 'Y':
            plt.savefig(path)
            print('File {} was overwriten'.format(path))
        else:
            print('File {} was not overwriten'.format(path))

save(df_expec, 'time', '<z>', '../plots/Relevant_data_task1.pdf')

### Task 3: Read in file `npop.t` and analyze correlations in the data

In [ ]:
 # read in npop.t
 df_2=read_in_df(filedir, filenames[2])

In [ ]:
# discard all columns with variance below a set threshold - we can consider them as constant
df_2=del_below(df_2,threshv)

Plot the remaining columns. Seaborn prefers "long format" (one column for all measurement values, one column to indicate the type) as input, whereas the cvs is in "wide format" (one column per measurement type).

In [ ]:
# plot ideally with seaborn
def plot_multiple(df, x, var_name, value_name):    
    df_2_melted=df_2.melt(x, var_name=var_name, value_name=value_name)
    sn.relplot(data=df_2_melted, kind='line', x=x, y=value_name, hue=var_name)  
    plt.savefig('')
    plt.show()

## Quantify the pairwise correlation in the data

- negative correlation: y values decrease for increasing x - large values of one feature correspond to small values of the other feature
- weak or no correlation: no trend observable, association between two features is hardly observable
- positive correlation: y values increase for decreasing x - small values of one feature correspond to small values of the other feature

Remember that correlation does not indicate causation - the reason that two features are associated can lie in their dependence on same factors.

Correlate the value pairs using Pearson's $r$. Pearson's $r$ is a measure of the linear relationship between features:

$r = \frac{\sum_i(x_i − \bar{x})(y_i − \bar{y})}{\sqrt{\sum_i(x_i − \bar{x})^2 \sum_i(y_i − \bar{y})^2}}$

Here, $\bar{x}$ and $\bar{y}$ indicate mean values. $i$ runs over the whole data set. For a positive correlation, $r$ is positive, and negative for a negative correlation, with minimum and maximum values of -1 and 1, indicating a perfectly linear relationship. Weakly or not correlated features are characterized by $r$-values close to 0.

Other measures of correlation that can be used are Spearman's rank (value pairs follow monotonic function) or Kendall's $\tau$ (measures ordinal association), but they do not apply here. You can also define measures yourself.

In [ ]:
# print the correlation matrix
#print(df_2.mean())
#print(df_2.loc[2, df_2.columns[2]])
#print(df_2.loc[2, df_2.columns[3]])

#def generate_nullmatrix(rows,columns):          #function to generate a 0 matrix with a specific number of rows and columns,
#    _list=[]                                     ended up not using this but leaving it in just in case
#    column=[]
#    for i in range(columns):
#        column.append(0)
#    for k in range(rows):
#         _list.append(column)
#    outp = np.array(_list)
#    return outp

def upper_sum(df,k,j):                #get the upper value, sum over the entire column k compared to column j
    outp=0
    for i in range(len(df.index)):
        outp = outp + (df.loc[i, df.columns[j]]-df.mean()[j])*(df.loc[i, df.columns[k]]-df.mean()[k])
    return outp

def lower_sum(df,k,j):                #get the lower value, apparently the length of some vector or smthg
    sum1=0                            #again compare given column k to given column j
    sum2=0
    for i in range(len(df.index)):
        sum1 = sum1 + (df.loc[i, df.columns[k]]- df.mean()[k])**2
    for i in range(len(df.index)):
        sum2 = sum2 + (df.loc[i, df.columns[j]]- df.mean()[j])**2
    outp = (sum1*sum2)**(1/2)
    return outp

def pear_values(df,j): 
    row_of_values=[]                        #get a row of values for a given first variable, comparing it to all others
    for k in range(len(df.columns)): 
            row_of_values.append(upper_sum(df,j,k)/lower_sum(df,j,k))
    return row_of_values

def pear_matrix(df):         #pull all rows through pear_values and merge them in one np.array with dimension [columns x columns]
    list_of_rows=[]            
    for i in range(len(df.columns)):
        list_of_rows.append(pear_values(df,i))
    matrix = np.array(list_of_rows) # -> this is the correlation matrix
    return matrix
            
#corr_matrix_array = pear_matrix(df_2)
#I don't think i got this right :) maybe i did now, whoop whoop!

corr_matrix = df_2.corr(method='pearson') #the easy way i guess, considerably better though as you get an idea what was compared
                                          #where my method only yields the pure matrix without such information
#print(corr_matrix)

The diagonal values tell us that each value is perfectly correlated with itself. We are not interested in the diagonal values and also not in the correlation with time. We also need to get rid of redundant entries. Finally, we need to find the value pairs that exhibit the highest linear correlation. We still want to know if it is positive or negative correlation, so we cannot get rid of the sign.

In [ ]:
# get rid of time column, lower triangular and diagonal entries of the correlation matrix
#corr_matrix_array = pear_matrix(df_2)

#def dropper(matrix):                    #this does exactly what i want it to do, but i lose the information on what was  
#    for i in range(len(matrix)):        #represented if i sort now
#        for k in range(len(matrix)):
#            if i == k: 
#                matrix[i,k] = 0
#            elif k == 0:
#                matrix[i,k] = 0
#            elif i > k :
#                matrix[i,k] = 0
#    return matrix
def correlation(df):
    corr_matrix = df.corr(method='pearson')
    print(corr_matrix)
    print('Drop lower triangular and diagonal values? Y/N')
    check = input()
    if check == 'Y':
        df_temp = pd.DataFrame(index=corr_matrix.index, columns=corr_matrix.columns)      
        for i in range(len(corr_matrix.columns)):                               
            for k in range(len(corr_matrix.index)):                             
                if i == k:                                              
                    df_temp.iloc[i, k] = 0         
                elif i > k:
                    df_temp.iloc[i, k] = 0
                else:
                    df_temp.iloc[i, k] = 1
        corr_matrix_reduced = corr_matrix*df_temp
        print(corr_matrix_reduced)
        return corr_matrix_reduced
    else:
        return corr_matrix 

def dropper_pandas(df):                                             #set values for diagonal and sub-diagonal elements to 0
    df_temp = pd.DataFrame(index=df.index, columns=df.columns)      #tried NaN, but ran into problems excluding these later
    for i in range(len(df.columns)):                                #whereas 0 can be easily checked against
        for k in range(len(df.index)):                              #the function creates a mask which is 0 for the diagonal
            if i == k:                                              #and sub-diagonal entries and 1 everywhere else, than 
                df_temp.iloc[i, k] = 0         #multiplies input dataframe with mask and returns product
            #elif k == 0:
             #   df_temp.iloc[i, k] = 0
            elif i > k:
                df_temp.iloc[i, k] = 0
            else:
                df_temp.iloc[i, k] = 1
    df = df*df_temp
    return df

#corr_matrix = df_2.corr(method='pearson')
#corr_matrix=dropper_pandas(corr_matrix)
#print(corr_matrix)

test = correlation(df_2)

# sort the remaing values according to their absolute value, but keep the sign

#how to sort but still now what was correlated? maybe somehow put index and columns name in list together with value and create list of such lists, then sort this list based on the values within the contained lists...
 
def array_of_entries_without_x(df, x):                #this creates an array based on such a list because this lets me understand 
    list_of_entries = []                              #how to index this catastrophe
    for i in range(len(df.index)):
        for k in range(len(df.columns)):        
            if df.index[i] != x :           #f*** time i guess
                if df.columns[k] != x :    
                    if df.loc[df.index[i], df.columns[k]] != 0 :        #can't ckeck against NaN apparently...
                        list_of_entries.append([df.index[i], df.columns[k], df.iloc[i, k]])
    list_of_entries = np.array(list_of_entries)
    return list_of_entries


def sort_array(array_in, axis, ):               #takes a np.array with n rows and m columns, sorts the entries of the column denominated by
    for i in range(1,len(array_in)):          #variable axis based on absolute value using insertion sort, keeps the sign, returns sorted 
        key_item = float(array_in[i,axis])    #np.array
        j = i-1
        while j >= 0 and abs(float(array_in[j,axis])) > abs(key_item):
            array_in[j+1,axis]=float(array_in[j,axis])
            j = j-1
        array_in[j+1, axis] = key_item
    return array_in
    
unsorted_array=array_of_entries_without_x(corr_matrix, x = 'time')
sorted_array=sort_array(unsorted_array, axis = 2)


In [ ]:
def get_values(df, exclude, sort, path):
    """[Extract values from correlation matrix, write to np.array.]

    Args:
        df ([DataFrame]): [Correlation Matrix]
        exclude ([String]): [Variable whos correlation is to be excluded]
        sort ([Boolean]): [Set to true for values to be sorted along a certain axis]

    Returns:
        [type]: [description]
    """
    list_of_entries = []                              
    for i in range(len(df.index)):
        for k in range(len(df.columns)):        
            if df.index[i] != x :           
                if df.columns[k] != x :    
                    if df.loc[df.index[i], df.columns[k]] != 0 :        
                        list_of_entries.append([df.index[i], df.columns[k], df.iloc[i, k]])
    list_of_entries = np.array(list_of_entries)     #list containing what was correlated and related values
    array_in = list_of_entries
    print(list_of_entries)

    if sort == True:
        print('Sort along which axis? Index: ')     
        axis = input()
        for i in range(1,len(array_in)):           
        key_item = float(array_in[i,axis])    
        j = i-1
        while j >= 0 and abs(float(array_in[j,axis])) > abs(key_item):
            array_in[j+1,axis]=float(array_in[j,axis])
            j = j-1
        array_in[j+1, axis] = key_item

        if Path(path).exists() == False:   #check if file exists already, ask before overwrite
            np.savetxt(path, array_in, fmt = '%s', header = 'Correlated : x -> y -> Pearson r(x,y) \n', delimiter = ' -> ', footer = '\nSorted using insertion sort based on absolute value \n')
        elif Path(path).exists() == True:
        print('{} already exists. Overwrite? Y/N'.format(path))
        check = input()
        if check == 'Y':
            np.savetxt(path, array_in, fmt = '%s', header = 'Correlated : x -> y -> Pearson r(x,y) \n', delimiter = ' -> ', footer = '\nSorted using insertion sort based on absolute value \n')
            print('File {} was overwriten'.format(path))
        else:
            print('File {} was not overwriten'.format(path))

    else:
        if Path(path).exists() == False:   #check if file exists already, ask before overwrite
            np.savetxt(path, array_in, fmt = '%s', header = 'Correlated : x -> y -> Pearson r(x,y) \n', delimiter = ' -> ')
        elif Path(path).exists() == True:
        print('{} already exists. Overwrite? Y/N'.format(path))
        check = input()
        if check == 'Y':
            np.savetxt(path, array_in, fmt = '%s', header = 'Correlated : x -> y -> Pearson r(x,y) \n', delimiter = ' -> ')
            print('File {} was overwriten'.format(path))
        else:
            print('File {} was not overwriten'.format(path))

get_values(test, 'time', True, '../data/Test1')
get_values(test, 'time', True, '../data/Test2')
get_values(test, 'time', True, '../data/Test3')

Note that the entries in the left column are not repeated if they do not change from the row above (so the fourth feature pair is MO3 and MO6).

### Task 4: Print the resulting data to a file

In [ ]:
# write to file
np.savetxt('../data/Correlated_npop.txt', sorted_array, fmt = '%s', header = 'Correlated : x -> y -> Pearson r(x,y) \n', delimiter = ' -> ', footer = '\nSorted using insertion sort based on absolute value \n')  #just some fancying around :D

### Task 5: Calculate the Euclidean distance (L2 norm) for the vectors in `table.dat`


The Euclidean distance measures the distance between to objects that are not points:

$d(p,q) = \sqrt{\left(p-q\right)^2}$

In this case, consider each of the columns in table.dat as a vector in Euclidean space, where column $r(x)$ and column $v(x)$ denote a pair of vectors that should be compared, as well as $r(y)$ and $v(y)$, and r(z) and v(z).

(Background: These are dipole moment components in different gauges, the length and velocity gauge.)

In [ ]:
# read in table.dat - I suggest reading it as a numpy array
# replace the NaNs by zero -> where are the NaNs? ah, there they are
table_data = read_in_df(filedir, filenames[4])
table_data=table_data.fillna(0)

Now calculate how different the vectors in column 2 are from column 3, column 4 from column 5, and column 6 from column 7.

In [ ]:
# calculate the Euclidean distance
def get_vectors(df, x, y):                                  #extract the interesting columns ("vectors") into separate dataframes
    df_out = pd.DataFrame(index=df.index, columns=[x, y])
    df_out[x] = df[x]
    df_out[y] = df[y]    
    return df_out

x_vectors=get_vectors(table_data, 'r(x)', 'v(x)')           
y_vectors=get_vectors(table_data, 'r(y)', 'v(y)')
z_vectors=get_vectors(table_data, 'r(z)', 'v(z)')

def get_distance(vectors):                                  #get distance between vectors by essentially creating a vector which points from one
    diff_summ = 0                                           #to the other (difference of input vectors) and than calculating the length of the 
    for i in range(len(vectors.index)):                     #new vector through (summ_components**2)**(1/2)
        diff_summ = diff_summ + (vectors.iloc[i, 0]- vectors.iloc[i, 1])**2
    distance = (diff_summ)**(1/2)
    return distance

distance_x = get_distance(x_vectors)
distance_y = get_distance(y_vectors)
distance_z = get_distance(z_vectors)


In [ ]:
# plot the result and save to a .pdf

#what, plot the values? just three points... I don't think I got this right
plt.plot(['x','y','z'],[distance_x,distance_y,distance_z], 'ob')
plt.ylabel('Distance r/v')
plt.xlabel('Axis')
plt.savefig('plots/The_distances_I_guess.pdf')
plt.show()

In [ ]:
# print the result to a file
save = np.array([['Distance r(x)/v(x)', distance_x], ['Distance r(y)/v(y)', distance_y], ['Distance r(z)/v(z)', distance_z]])

np.savetxt('data/saved_distances_table.txt', save, fmt='%s' , delimiter = ' : ')

# Numerical analysis

Analyze the data using autocorrelation functions and discrete Fourier transforms. Plot your results.

In [ ]:
import numpy as np
def read_in_np(filedir, filename):
    name = '{}{}'.format(filedir, filename)
    print('Reading from file {} - numpy'.format(name))
    data = np.loadtxt(name, skiprows=1)
    data = data.T
    return data

### Task 1: Read in `efield.t` and Fourier-transform relevant columns

In [ ]:
efield = read_in_np('data/' , 'efield.t')
#print(efield)

Here we are interested in column 2 since the others are constant.

In [ ]:
# discard the columns with variance below threshold - these are considered constant

efield_y=efield[2]
#print(efield_y)

In [ ]:
# discrete Fourier transform of the remaining column: You only need the real frequencies
signal = np.fft.rfft(efield_y)
signal = signal.real
n = efield[0].size
freq = np.fft.rfftfreq(n) #hier rfft damit beide gleich lang sind mit fft kann nicht geplottet werde
print(efield_dft)
print(freq)
print(efield_dft.size) #überprüfen ob beide gleich lang sind
print(freq.size)


### Task 2: Generate a plot of your results to be saved as pdf.

In [ ]:
# plot your results
import matplotlib.pyplot as plt
#print(len(efield[0]))
#print(len(efield_dft))
#plt.hist(efield_dft)
#plt.xlabel('Frequenz')
#plt.ylabel('Anzahl')
#plt.savefig('efield_dft_hist.pdf')
plt.plot( freq , efield_dft ) #plot freq gegn signal
plt.savefig('efield_dft.pdf') #speichern

### Task 3: Calculate the autocorrelation function from nstate_i.t
The autocorrelation function measures how correlated subsequent vectors are with an initial vector; ie. 

$\Psi_{corr} = \langle \Psi(t=0) | \Psi(t) \rangle = \int_0^{tfin} \Psi(0)^* \Psi(t) dt$

Since we are in a numerical representation, the integral can be replaced with a sum; and the given vectors are already normalized.

In [ ]:
# read in as numpy array
nstate = read_in_np('data/' , 'nstate_i.t')
print(nstate)


In [ ]:
# store the time column (column 0) in a vector and drop from array 
time = nstate[0]
print(time)

In [ ]:
# correct the data representation: this is in fact a complex matrix
# the real part of each matrix column is contained in numpy array column 0, 2, 4, 6, ...
# the imaginary part of each matrix column is contained in numpy array column 1, 3, 5, 7, ...
# convert the array that was read as dtype=float into a dtype=complex array
fct = np.delete(nstate,[0],axis=0) #zeit aus der matrix löschen da hierfür nicht relevant
real = fct[0::2]
imag = fct[1::2]
comp_fct = real + 1j*imag #zusammensetzen der beiden Teile 
print(comp_fct)

In [ ]:
# for the autocorrelation function, we want the overlap between the first vector at time 0 and all 
# subsequent vectors at later times - the sum of the product of initial and subsequent vectors for each time step

#d#def calc_auto(comp_fct):
  #  for i in range(0,len(comp_fct[0])):
   #     aucofu[i] = np.sum(comp_fct[:,0]*np.conjugate(comp_fct[:,i]))  
   #      aucofu = np.append(comp_fct[:,0],dtype = complex) geht irgendwie nicht :( ahhhh wiesooooo
    #return aucofu
def rechne_autofct(comp_fct):
    autofct = np.zeros(len(comp_fct[0]), dtype = complex) #leere (null) matrix mit länge von komplexer 
    for i in range(0,len(comp_fct[0])):
        autofct[i] = np.sum(comp_fct[:,0]*np.conjugate(comp_fct[:,i])) #funktion von oben 
    return autofct
    

In [ ]:
autofct = rechne_autofct(comp_fct)
print(autofct)

### Task 4: Generate a plot of your results to be saved as pdf.

In [ ]:
autofct_reel = np.absolute(autofct**2) #da das sonst imaginäre ist 
plt.plot(autofct_reel)
plt.savefig('autofct_plot.pdf')

### Task 5: Discrete Fourier transform of the autocorrelation function

In [ ]:
# discrete Fourier-transform the autocorrelation function - now we need all frequency components, 
# also the negative ones
autofct_dft = np.fft.fft(autofct)
time_dft = np.fft.fftfreq(len(time))
print(time.size)
print(autofct_dft.size) #größe überprüfen 

### Task 6: Generate a plot of your results to be saved as pdf.

In [ ]:
# plot the power spectrum (abs**2)
time_reel = np.absolute(time_dft**2)
autofct_reel = np.absolute(autofct_dft) #bin nicht sicher ob man das somachen soll...
plt.plot(autofct_reel , time_reel)
plt.savefig('end_plot.pdf')